# 天猫订单数据分析

## 数据导入与处理

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('tmall_order_report.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.columns = data.columns.str.strip() # 去除列名两端的空格
data.columns

In [ ]:
data[data.duplicated()].count() # 重复行数

In [ ]:
data.isnull().sum() # 付款时间存在空值表明有订单未支付

## 数据分析可视化

1. 整体情况

In [ ]:
result = {}
result['总订单数'] = data['订单编号'].count()
result['已完成订单数'] = data['订单编号'][data['订单付款时间'].notnull()].count()
result['未付款订单数'] = data['订单编号'][data['订单付款时间'].isnull()].count()
result['退款订单数'] = data['订单编号'][data['退款金额'] > 0].count()
result['总订单金额'] = data['总金额'][data['订单付款时间'].notnull()].sum()
result['总退单金额'] = data['退款金额'][data['订单付款时间'].notnull()].sum()
result['平均订单金额'] = data['总金额'][data['订单付款时间'].notnull()].mean()
result['最高订单金额'] = data['总金额'][data['订单付款时间'].notnull()].max()
result['最低订单金额'] = data['总金额'][data['订单付款时间'].notnull()].min()
result['总实际收入金额'] = data['买家实际支付金额'][data['订单付款时间'].notnull()].sum()

In [ ]:
result

In [ ]:
from pyecharts.charts import Map, Bar, Line
from pyecharts import options as opts
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
from pyecharts.faker import Faker

In [ ]:
table = Table()
headers = ['总订单数', '总订单金额', '已完成订单数', '总实际收入金额', '未付款订单数', '退款订单数', '总退单金额', '平均订单金额', '最高订单金额', '最低订单金额', '成交率', '退货率']

rows = [
    [
        result['总订单数'],
        f"{result['总订单金额']/10000:.2f} 万",
        result['已完成订单数'],
        f"{result['总实际收入金额']/10000:.2f} 万",
        result['未付款订单数'],
        result['退款订单数'],
        f"{result['总退单金额']/10000:.2f} 万",
        f"{result['平均订单金额']:.2f} 元",
        f"{result['最高订单金额']:.2f} 元",
        f"{result['最低订单金额']:.2f} 元",
        f"{result['已完成订单数']/result['总订单数']*100:.2f} %",
        f"{result['退款订单数']/result['已完成订单数']*100:.2f} %"
    ]
]
table.add(headers, rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title="天猫订单数据分析")
)
table.render_notebook()

2. 各省订单量

In [ ]:
data.收货地址.value_counts()

In [ ]:
# 规范省份名
def rename(p):
    if p in ['北京', '天津', '上海', '重庆']:
        return p + '市'
    return p
data.收货地址 = data.收货地址.map(rename)
data.收货地址.value_counts()

In [ ]:
data[data.订单付款时间.notnull()]

In [ ]:
# 统计各省订单数量
res = data[data.订单付款时间.notnull()]
res = res.收货地址.value_counts()
order_data = res.to_dict()
order_data

In [ ]:
c = (
    Map(init_opts = opts.InitOpts(width = '1000px', height = '600px'))
    .add('各省订单量',
         [list(i) for i in zip(order_data.keys(), order_data.values())],
         'china',
         is_map_symbol_show=False
     )
    .set_series_opts(label_opts = opts.LabelOpts(is_show = True))
    .set_global_opts(
        title_opts = opts.TitleOpts(title = '天猫订单各省订单量'),
        visualmap_opts = opts.VisualMapOpts(
            max_ = 3100,
            is_piecewise = True, # 定义图例诶分段式，默认为连续式
            pos_top = 'middle', # 分段位置
            pos_left = 'left',
            orient = 'vertical',
            split_number = 10
        )
    )
)
c.render_notebook()

3. 时间序列分析

In [ ]:
data['订单创建时间'] = pd.to_datetime(data.订单创建时间)
data['订单付款时间'] = pd.to_datetime(data.订单付款时间)
data.head()

In [ ]:
# 格式化日期格式
order_time = data.订单创建时间.map(lambda x: x.strftime('%Y-%m-%d'))
data['order_add_time'] = order_time
# 统计每天下单数量
cnt = pd.pivot_table(data, index = 'order_add_time', values = '订单编号', aggfunc = 'count')
result = cnt.to_dict()['订单编号']

In [ ]:
# 每日下单数量折线统计图
c = (
    Line()
    .add_xaxis(list(result.keys()))
    .add_yaxis('下单数量', list(result.values()))
    .set_global_opts(
        title_opts = opts.TitleOpts(title = '天猫订单每日下单数量'),
    )
    .set_series_opts(
        Label_opts = opts.LabelOpts(is_show = False),
        markpoint_opts = opts.MarkPointOpts(
            data = [
                opts.MarkPointItem(type_ = 'max')
            ]
        )
    )
    .set_global_opts(
        title_opts = opts.TitleOpts(title = '天猫订单每日下单数量')
    )
)
c.render_notebook()

In [ ]:
# 每小时订单数量可视化
order_add_time2 = data.订单创建时间.map(lambda x: x.strftime('%H'))
# 统计每小时下单数量
cnt2 = data.groupby(order_add_time2).agg({'订单编号' : 'count'}).to_dict()['订单编号']

c = (
    Bar()
    .add_xaxis(list(cnt2.keys()))
    .add_yaxis('订单量', list(cnt2.values()))
    .set_series_opts(
        label_opts = opts.LabelOpts(is_show = False),
        markpoint_opts = opts.MarkPointOpts(
            data = [
                opts.MarkPointItem(type_ = 'max')
            ]
        )
    )
    .set_global_opts(
        title_opts = opts.TitleOpts(title = '每小时订单量')
    )
)
c.render_notebook()

# 双十一美妆数据

In [ ]:
data = pd.read_csv('双十一淘宝美妆数据.csv')
data

In [ ]:
data.info()
data.shape

In [ ]:
# 查看数据
data.describe()

In [ ]:
data.sale_count.mode()
data.comment_count.mode()

In [ ]:
# 查看每家店情况
data['店名'].value_counts()

In [ ]:
# 去重
data = data.drop_duplicates(inplace = False)
data

In [ ]:
# 重置索引
data.reset_index(inplace = True, drop = True)
data

In [ ]:
# 处理缺失值
data.isnull().any()

In [ ]:
# 填充缺失值
data = data.fillna(0) # NaN用0填充
data.isnull().any()

## 提取有效信息

In [ ]:
import jieba

In [ ]:
title_cut = []
for i in data['title']:
    title_cut.append(jieba.lcut(i))
data['item_name_cut'] = title_cut
data[['title', 'item_name_cut']].head()

In [ ]:
# 给商品添加分类
sub_type = []
main_type = []
category_config_map = {
    '乳液': ('护肤品', '乳液类'),
    '美白乳': ('护肤品', '乳液类'),
    '润肤乳': ('护肤品', '乳液类'),
    '凝乳': ('护肤品', '乳液类'),
    "柔肤液'": ('护肤品', '乳液类'),
    '亮肤乳': ('护肤品', '乳液类'),
    '菁华乳': ('护肤品', '乳液类'),
    '眼霜': ('护肤品', '眼部护理'),
    '眼部精华': ('护肤品', '眼部护理'),
    '洗面': ('护肤品', '清洁类'),
    '洁面': ('护肤品', '清洁类'),
    '清洁': ('护肤品', '清洁类'),
    '卸妆': ('护肤品', '清洁类'),
    '洁颜': ('护肤品', '清洁类'),
    '洗颜': ('护肤品', '清洁类'),
    '去角质': ('护肤品', '清洁类'),
    '化妆水': ('护肤品', '化妆水'),
    '爽肤水': ('护肤品', '化妆水'),
    '柔肤水': ('护肤品', '化妆水'),
    '补水露': ('护肤品', '化妆水'),
    '凝露': ('护肤品', '化妆水'),
    '柔肤液': ('护肤品', '化妆水'),
    '精粹水': ('护肤品', '化妆水'),
    '亮肤水': ('护肤品', '化妆水'),
    '润肤水': ('护肤品', '化妆水'),
    '保湿水': ('护肤品', '化妆水'),
    '菁华水': ('护肤品', '化妆水'),
    '保湿喷雾': ('护肤品', '化妆水'),
    '面霜': ('护肤品', '面霜类'),
    '日霜': ('护肤品', '面霜类'),
    '晚霜': ('护肤品', '面霜类'),
    '柔肤霜': ('护肤品', '面霜类'),
    '滋润霜': ('护肤品', '面霜类'),
    '保湿霜': ('护肤品', '面霜类'),
    '凝霜': ('护肤品', '面霜类'),
    '日间霜': ('护肤品', '面霜类'),
    '晚间霜': ('护肤品', '面霜类'),
    '乳霜': ('护肤品', '面霜类'),
    '修护霜': ('护肤品', '面霜类'),
    '亮肤霜': ('护肤品', '面霜类'),
    '底霜': ('护肤品', '面霜类'),
    '精华液': ('护肤品', '精华类'),
    '精华水': ('护肤品', '精华类'),
    '精华露': ('护肤品', '精华类'),
    '防晒霜': ('护肤品', '防晒类'),
    '唇釉': ('化妆品', '口红类'),
    '口红': ('化妆品', '口红类'),
    '散粉': ('化妆品', '底妆类'),
    '蜜粉': ('化妆品', '底妆类'),
    '粉底液': ('化妆品', '底妆类'),
    '定妆粉 ': ('化妆品', '底妆类'),
    '气垫': ('化妆品', '底妆类'),
    '粉饼': ('化妆品', '底妆类'),
    'BB': ('化妆品', '底妆类'),
    'CC': ('化妆品', '底妆类'),
    '遮瑕': ('化妆品', '底妆类'),
    '粉霜': ('化妆品', '底妆类'),
    '粉底膏': ('化妆品', '底妆类'),
    '眉粉': ('化妆品', '眼部彩妆'),
    '染眉膏': ('化妆品', '眼部彩妆'),
    '眼线': ('化妆品', '眼部彩妆'),
    '眼影': ('化妆品', '眼部彩妆'),
    '鼻影': ('化妆品', '修容类'),
    '修容粉': ('化妆品', '修容类'),
    '高光': ('化妆品', '修容类')
}

In [ ]:
# 替换数据
for i in range(len(data)):
    exist = False
    for temp in data.item_name_cut[i]:
        if temp in category_config_map:
            sub_type.append(category_config_map.get(temp)[1])
            main_type.append(category_config_map.get(temp)[0])
            exist = True
            break
    if not exist:
        sub_type.append('其他')
        main_type.append('其他')
data['sub_type'] = sub_type
data['main_type'] = main_type

In [ ]:
data[['title', 'sub_type', 'main_type']]

In [ ]:
data.groupby('sub_type').agg(总数 = ('id', 'count')).sort_values(by = '总数')

In [ ]:
data['main_type'].value_counts()

## 男士专用处理

In [ ]:
# 将是否为男塾专用新增一列
gender = []
for i in range(len(data)):
    if '男' in data.item_name_cut[i]:
        gender.append('是')
    elif '男士' in data.item_name_cut[i]:
        gender.append('是')
    elif '男生' in data.item_name_cut[i]:
        gender.append('是')
    else:
        gender.append('否')
data['是否男士专用'] = gender
data['是否男士专用'].value_counts()

In [ ]:
# 新增销售额、购买日期
data['销售额'] = data['sale_count'] + data['price']
data['update_time'] = pd.to_datetime(data['update_time'])
data

In [ ]:
# 将时间设为新的index
data = data.set_index('update_time')
data

In [ ]:
# 新增时间’天‘为新列
data['day'] = data.index.day
# 删除中文分词的一列
del data['item_name_cut']
data

## Data Analyst

In [ ]:
import pygwalker as pyg

In [ ]:
vis_spec = r"""
{
    "config": [
        {
            "config": {
                "defaultAggregated": true,
                "geoms": ["line"],
                "coordSystem": "generic",
                "limit": -1
            },
            "encodings": {
                "dimensions": [
                    {
                        "dragId": "gw_ucT8",
                        "fid": "id",
                        "name": "id",
                        "basename": "id",
                        "semanticType": "nominal",
                        "analyticType": "dimension"
                    },
                    {
                        "dragId": "gw_JmBE",
                        "fid": "title",
                        "name": "title",
                        "basename": "title",
                        "semanticType": "nominal",
                        "analyticType": "dimension"
                    },
                    {
                        "dragId": "gw_yKNY",
                        "fid": "店名",
                        "name": "店名",
                        "basename": "店名",
                        "semanticType": "nominal",
                        "analyticType": "dimension"
                    },
                    {
                        "dragId": "gw_4Xwb",
                        "fid": "sub_type",
                        "name": "sub_type",
                        "basename": "sub_type",
                        "semanticType": "nominal",
                        "analyticType": "dimension"
                    },
                    {
                        "dragId": "gw_YaB3",
                        "fid": "main_type",
                        "name": "main_type",
                        "basename": "main_type",
                        "semanticType": "nominal",
                        "analyticType": "dimension"
                    },
                    {
                        "dragId": "gw_merI",
                        "fid": "是否男士专用",
                        "name": "是否男士专用",
                        "basename": "是否男士专用",
                        "semanticType": "nominal",
                        "analyticType": "dimension"
                    },
                    {
                        "dragId": "gw_7JZo",
                        "fid": "day",
                        "name": "day",
                        "basename": "day",
                        "semanticType": "quantitative",
                        "analyticType": "dimension"
                    },
                    {
                        "dragId": "gw_mea_key_fid",
                        "fid": "gw_mea_key_fid",
                        "name": "Measure names",
                        "analyticType": "dimension",
                        "semanticType": "nominal"
                    }
                ],
                "measures": [
                    {
                        "dragId": "gw_xHoS",
                        "fid": "price",
                        "name": "price",
                        "basename": "price",
                        "analyticType": "measure",
                        "semanticType": "quantitative",
                        "aggName": "sum"
                    },
                    {
                        "dragId": "gw__iq3",
                        "fid": "sale_count",
                        "name": "sale_count",
                        "basename": "sale_count",
                        "analyticType": "measure",
                        "semanticType": "quantitative",
                        "aggName": "sum"
                    },
                    {
                        "dragId": "gw_mut8",
                        "fid": "comment_count",
                        "name": "comment_count",
                        "basename": "comment_count",
                        "analyticType": "measure",
                        "semanticType": "quantitative",
                        "aggName": "sum"
                    },
                    {
                        "dragId": "gw_BClV",
                        "fid": "销售额",
                        "name": "销售额",
                        "basename": "销售额",
                        "analyticType": "measure",
                        "semanticType": "quantitative",
                        "aggName": "sum"
                    },
                    {
                        "dragId": "gw_count_fid",
                        "fid": "gw_count_fid",
                        "name": "Row count",
                        "analyticType": "measure",
                        "semanticType": "quantitative",
                        "aggName": "sum",
                        "computed": true,
                        "expression": {
                            "op": "one",
                            "params": [],
                            "as": "gw_count_fid"
                        }
                    },
                    {
                        "dragId": "gw_mea_val_fid",
                        "fid": "gw_mea_val_fid",
                        "name": "Measure values",
                        "analyticType": "measure",
                        "semanticType": "quantitative",
                        "aggName": "sum"
                    }
                ],
                "rows": [
                    {
                        "dragId": "gw_SMMo",
                        "fid": "sale_count",
                        "name": "sale_count",
                        "basename": "sale_count",
                        "analyticType": "measure",
                        "semanticType": "quantitative",
                        "aggName": "sum"
                    }
                ],
                "columns": [
                    {
                        "dragId": "gw_LPYL",
                        "fid": "day",
                        "name": "day",
                        "basename": "day",
                        "semanticType": "quantitative",
                        "analyticType": "dimension"
                    }
                ],
                "color": [],
                "opacity": [],
                "size": [],
                "shape": [],
                "radius": [],
                "theta": [],
                "longitude": [],
                "latitude": [],
                "geoId": [],
                "details": [],
                "filters": [],
                "text": []
            },
            "layout": {
                "showActions": false,
                "showTableSummary": false,
                "stack": "stack",
                "interactiveScale": false,
                "zeroScale": true,
                "size": {
                    "mode": "fixed",
                    "width": 496,
                    "height": 344
                },
                "format": {},
                "geoKey": "name",
                "resolve": {
                    "x": false,
                    "y": false,
                    "color": false,
                    "opacity": false,
                    "shape": false,
                    "size": false
                }
            },
            "visId": "gw__8vg",
            "name": "Chart 1"
        }
    ],
    "chart_map": {},
    "workflow_list": [
        {
            "workflow": [
                {
                    "type": "view",
                    "query": [
                        {
                            "op": "aggregate",
                            "groupBy": ["day"],
                            "measures": [
                                {
                                    "field": "sale_count",
                                    "agg": "sum",
                                    "asFieldKey": "sale_count_sum"
                                }
                            ]
                        }
                    ]
                }
            ]
        }
    ],
    "version": "0.4.6"
}
"""
pyg.walk(data, spec=vis_spec)

分析结果：显示了销量和销量额随着时间变化的情况

1、销量高峰出现在9号，说明可能双十一商家提前预热，已经有很大优惠，消费者购物欲望强烈

2、销量低峰出现在11号，说明消费者可能已经提前买好，或是避开网络高峰

3、11号后销量走高，可能是商家持续打折，消费者还有购物余热

## 品牌分析

### SKU分析

In [ ]:
vis_spec = r"""
{
    "config": [
        {
            "config": {
                "defaultAggregated": true,
                "geoms": ["auto"],
                "coordSystem": "generic",
                "limit": -1
            },
            "encodings": {
                "dimensions": [
                    {"dragId": "gw_lygP", "fid": "id", "name": "id", "basename": "id", "semanticType": "nominal", "analyticType": "dimension"},
                    {"dragId": "gw_Uq4D", "fid": "title", "name": "title", "basename": "title", "semanticType": "nominal", "analyticType": "dimension"},
                    {"dragId": "gw_ep0t", "fid": "店名", "name": "店名", "basename": "店名", "semanticType": "nominal", "analyticType": "dimension"},
                    {"dragId": "gw_3Xze", "fid": "sub_type", "name": "sub_type", "basename": "sub_type", "semanticType": "nominal", "analyticType": "dimension"},
                    {"dragId": "gw_W8JR", "fid": "main_type", "name": "main_type", "basename": "main_type", "semanticType": "nominal", "analyticType": "dimension"},
                    {"dragId": "gw_c7IW", "fid": "是否男士专用", "name": "是否男士专用", "basename": "是否男士专用", "semanticType": "nominal", "analyticType": "dimension"},
                    {"dragId": "gw_64Hk", "fid": "day", "name": "day", "basename": "day", "semanticType": "quantitative", "analyticType": "dimension"},
                    {"dragId": "gw_mea_key_fid", "fid": "gw_mea_key_fid", "name": "Measure names", "analyticType": "dimension", "semanticType": "nominal"}
                ],
                "measures": [
                    {"dragId": "gw_7u7q", "fid": "price", "name": "price", "basename": "price", "analyticType": "measure", "semanticType": "quantitative", "aggName": "sum"},
                    {"dragId": "gw_O6GA", "fid": "sale_count", "name": "sale_count", "basename": "sale_count", "analyticType": "measure", "semanticType": "quantitative", "aggName": "sum"},
                    {"dragId": "gw_XiQT", "fid": "comment_count", "name": "comment_count", "basename": "comment_count", "analyticType": "measure", "semanticType": "quantitative", "aggName": "sum"},
                    {"dragId": "gw_QKei", "fid": "销售额", "name": "销售额", "basename": "销售额", "analyticType": "measure", "semanticType": "quantitative", "aggName": "sum"},
                    {"dragId": "gw_count_fid", "fid": "gw_count_fid", "name": "Row count", "analyticType": "measure", "semanticType": "quantitative", "aggName": "sum", "computed": true, "expression": {"op": "one", "params": [], "as": "gw_count_fid"}},
                    {"dragId": "gw_mea_val_fid", "fid": "gw_mea_val_fid", "name": "Measure values", "analyticType": "measure", "semanticType": "quantitative", "aggName": "sum"}
                ],
                "rows": [
                    {"dragId": "gw_ug94", "fid": "gw_count_fid", "name": "Row count", "analyticType": "measure", "semanticType": "quantitative", "aggName": "sum", "computed": true, "expression": {"op": "one", "params": [], "as": "gw_count_fid"}}
                ],
                "columns": [
                    {"dragId": "gw_vTfJ", "fid": "店名", "name": "店名", "basename": "店名", "semanticType": "nominal", "analyticType": "dimension", "sort": "descending"}
                ],
                "color": [
                    {"dragId": "gw_8f2j", "fid": "店名", "name": "店名", "basename": "店名", "semanticType": "nominal", "analyticType": "dimension"}
                ],
                "opacity": [],
                "size": [],
                "shape": [],
                "radius": [],
                "theta": [],
                "longitude": [],
                "latitude": [],
                "geoId": [],
                "details": [],
                "filters": [],
                "text": []
            },
            "layout": {
                "showActions": false,
                "showTableSummary": false,
                "stack": "stack",
                "interactiveScale": false,
                "zeroScale": true,
                "size": {"mode": "fixed", "width": 521, "height": 415},
                "format": {},
                "geoKey": "name",
                "resolve": {"x": false, "y": false, "color": false, "opacity": false, "shape": false, "size": false}
            },
            "visId": "gw_Z68w",
            "name": "Chart 1"
        }
    ],
    "chart_map": {},
    "workflow_list": [
        {
            "workflow": [
                {
                    "type": "transform",
                    "transform": [
                        {"key": "gw_count_fid", "expression": {"op": "one", "params": [], "as": "gw_count_fid"}}
                    ]
                },
                {
                    "type": "view",
                    "query": [
                        {"op": "aggregate", "groupBy": ["店名"], "measures": [{"field": "gw_count_fid", "agg": "sum", "asFieldKey": "gw_count_fid_sum"}]}
                    ]
                }
            ]
        }
    ],
    "version": "0.4.6"
}
"""

pyg.walk(data, spec=vis_spec)

分析结果：显示了每个品牌的SKU

悦诗风吟的商品数量最多，其次是佰草集和欧莱雅，蜜丝佛陀最少

### 各品牌的类别占比

In [ ]:
vis_spec = r"""{"config":[{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1,"folds":["price"]},"encodings":{"dimensions":[{"dragId":"gw_etTu","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_LLa9","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw__MED","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_SHBe","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_s--Z","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw__Fzo","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_uS76","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_-EKC","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_g5Im","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_FueH","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_XJ_E","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_rKa4","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_muUC","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_yndC","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"columns":[{"dragId":"gw_vUPl","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"}],"color":[{"dragId":"gw_Cbnp","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[],"shape":[],"radius":[{"dragId":"gw_yzGv","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}}],"theta":[{"dragId":"gw_dCPS","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[{"dragId":"gw_FRMq","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["其他"]}}],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"normalize","interactiveScale":false,"zeroScale":true,"size":{"mode":"auto","width":440,"height":433},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false},"scaleIncludeUnmatchedChoropleth":false,"colorPalette":"","useSvg":false,"scale":{"opacity":{},"size":{}},"primaryColor":"rgba(142,209,252,1)"},"visId":"gw_tjOH","name":"Chart 1"},{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_E0_k","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_hTLZ","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_iAJj","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_SB19","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_Mhhf","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_ZxAX","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_biM-","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_66ZV","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_unS-","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_jA_O","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_j7oc","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_QH6P","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_AwH1","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_h9XM","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"columns":[{"dragId":"gw_nmJD","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"}],"color":[{"dragId":"gw_KK70","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[{"dragId":"gw_mDDC","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["其他"]}}],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"normalize","interactiveScale":false,"zeroScale":true,"size":{"mode":"auto","width":520,"height":417},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_v0f1","name":"Chart 2"}],"chart_map":{},"workflow_list":[{"workflow":[{"type":"filter","filters":[{"fid":"sub_type","rule":{"type":"not in","value":["其他"]}}]},{"type":"transform","transform":[{"key":"gw_count_fid","expression":{"op":"one","params":[],"as":"gw_count_fid"}}]},{"type":"view","query":[{"op":"aggregate","groupBy":["店名","main_type"],"measures":[{"field":"gw_count_fid","agg":"sum","asFieldKey":"gw_count_fid_sum"},{"field":"sale_count","agg":"sum","asFieldKey":"sale_count_sum"},{"field":"销售额","agg":"sum","asFieldKey":"销售额_sum"}]}]}]},{"workflow":[{"type":"filter","filters":[{"fid":"sub_type","rule":{"type":"not in","value":["其他"]}}]},{"type":"transform","transform":[{"key":"gw_count_fid","expression":{"op":"one","params":[],"as":"gw_count_fid"}}]},{"type":"view","query":[{"op":"aggregate","groupBy":["店名","sub_type"],"measures":[{"field":"gw_count_fid","agg":"sum","asFieldKey":"gw_count_fid_sum"},{"field":"sale_count","agg":"sum","asFieldKey":"sale_count_sum"},{"field":"销售额","agg":"sum","asFieldKey":"销售额_sum"}]}]}]}],"version":"0.4.6"}"""
pyg.walk(data, spec=vis_spec)

分析结果：

Chart1：显示了除去其他产品，各品牌销售的护肤品和化妆品的数量比例、销量比例、销量额比例

1、除去其他产品，各品牌主打的销售类别有所不同：美宝莲、植村秀、蜜丝佛陀主打化妆品，其余品牌主打护肤品

2、娇兰、雪花秀虽然化妆品品类少，但是却占了销量和销量额的大头

Chart2：显示了除去其他产品，各品牌销售的各类别的数量比例、销量比例、销量额比例

1、主打化妆品的品牌主要销售底妆类产品，主打护肤品的品牌主要销售清洁类、面霜和化妆水

2、娇兰口红品类虽然数量较少，销售额占比最大；雪花秀底妆类产品数量虽较少，但销售占比最大

### 各品牌价格

In [ ]:
vis_spec = r"""{"config":[{"config":{"defaultAggregated":false,"geoms":["boxplot"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_IOsW","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_twqg","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_AURd","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw__NYq","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_OMFU","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_2L8P","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_pYnT","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_GILd","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_iVEz","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_RNn-","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_D8Tx","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_JitA","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"columns":[{"dragId":"gw_HRXu","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension","sort":"descending"}],"color":[],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"none","interactiveScale":false,"zeroScale":true,"size":{"mode":"full","width":677,"height":590},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false},"scaleIncludeUnmatchedChoropleth":false,"colorPalette":"","useSvg":false,"scale":{"opacity":{"domainMax":3000},"size":{"domainMax":3000}},"primaryColor":"rgba(247,141,167,1)"},"visId":"gw_mlcG","name":"Chart 1"},{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1,"folds":[]},"encodings":{"dimensions":[{"dragId":"gw_QL1i","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_BxtD","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_whgW","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_ObEu","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_XK03","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_LdgO","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_S8O1","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_xyoE","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_wVZL","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_6eB-","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_Jnq_","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_58CV","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"columns":[{"dragId":"gw_PvTk","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension","sort":"descending"}],"color":[],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[{"dragId":"gw_da3p_auto","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"filters":[],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"auto","width":320,"height":200},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_7Twa","name":"Chart 2"},{"config":{"defaultAggregated":true,"geoms":["circle"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_YqGq","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_oJN_","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_a6fD","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_cxx0","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_R9Vl","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_rq5H","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_O3S-","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_Vyox","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_5fku","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_NgD9","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_W3p4","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_QcIW","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"columns":[{"dragId":"gw_i6WB","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"color":[{"dragId":"gw_hlNh","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[{"dragId":"gw__Rk_","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"fixed","width":732,"height":526},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_mxC6","name":"Chart 3"}],"chart_map":{},"workflow_list":[{"workflow":[{"type":"view","query":[{"op":"raw","fields":["店名","price"]}]}]},{"workflow":[{"type":"view","query":[{"op":"aggregate","groupBy":["店名"],"measures":[{"field":"price","agg":"mean","asFieldKey":"price_mean"}]}]}]},{"workflow":[{"type":"view","query":[{"op":"aggregate","groupBy":["店名"],"measures":[{"field":"sale_count","agg":"mean","asFieldKey":"sale_count_mean"},{"field":"销售额","agg":"mean","asFieldKey":"销售额_mean"},{"field":"price","agg":"mean","asFieldKey":"price_mean"}]}]}]}],"version":"0.4.6"}"""
pyg.walk(data, spec=vis_spec)

分析结果：

Chart1：显示了各品牌的箱型图

Chart2：显示了各品牌的平均价格

1、娇兰、SKII、雪花秀、雅诗兰黛、兰蔻、资生堂作为国际大牌，产品价格很高

2、佰草集是价格最高的国货，美加净是价格最低的国货

Chart3：显示了各品牌的销量和销量额的关系

1、销售量和销售额呈现正相关

2、销量和销量额与价格有很重要的关系

3、相宜本草、美宝莲、蜜丝佛陀销量和销售额位居前三，且平均价格居中

## 各品牌销量与销售额

### 各品牌热度

In [ ]:
vis_spec = r"""{"config":[{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_EL_S","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_gBLj","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_Ct4E","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_-P3T","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_-caO","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_5PMe","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_xlum","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_hoNj","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_Vnl2","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_Lg1Q","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_aPev","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_gkMF","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"columns":[{"dragId":"gw_XDE9","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension","sort":"descending"}],"color":[],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"fixed","width":648,"height":481},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_godC","name":"Chart 1"},{"config":{"defaultAggregated":true,"geoms":["circle"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_bNJE","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_Kgrd","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_X7Ws","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_7c0Z","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_NpG8","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_pmnD","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_pFri","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_hGDz","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_A3VG","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_B7--","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_816A","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_dwIx","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"columns":[{"dragId":"gw_vXG8","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"color":[{"dragId":"gw_iZTS","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"}],"opacity":[{"dragId":"gw_75iF","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"size":[{"dragId":"gw_2mbk","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[{"dragId":"gw_wxVF","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"filters":[],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"none","interactiveScale":false,"zeroScale":true,"size":{"mode":"full","width":713,"height":578},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false},"scaleIncludeUnmatchedChoropleth":false,"colorPalette":"","useSvg":false,"scale":{"opacity":{},"size":{}},"primaryColor":"rgba(247,141,167,1)"},"visId":"gw_aUJy","name":"Chart 2"}],"chart_map":{},"workflow_list":[{"workflow":[{"type":"view","query":[{"op":"aggregate","groupBy":["店名"],"measures":[{"field":"comment_count","agg":"mean","asFieldKey":"comment_count_mean"}]}]}]},{"workflow":[{"type":"view","query":[{"op":"aggregate","groupBy":["店名"],"measures":[{"field":"sale_count","agg":"mean","asFieldKey":"sale_count_mean"},{"field":"comment_count","agg":"mean","asFieldKey":"comment_count_mean"},{"field":"price","agg":"mean","asFieldKey":"price_mean"}]}]}]}],"version":"0.4.6"}"""
pyg.walk(data, spec=vis_spec)

分析结果：

Chart1：各品牌的评论数

美宝莲的平均商品评论最多，热度最高

Chart2：销量和评论数（热度）之间的关系

1、 热度与销量呈现一定正相关

2、 价格低的品牌热度和销量相对较高，价格高的品牌热度和销量相对较低，说明价格在热度和销量中有一定影响

### 各类别分析

*每个类别的销量和销量额*

In [ ]:
vis_spec = r"""{"config":[{"config":{"defaultAggregated":true,"geoms":["arc"],"coordSystem":"generic","limit":-1,"folds":["price"]},"encodings":{"dimensions":[{"dragId":"gw_7CTD","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_Z-5w","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_IGZj","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_kiwQ","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_gUC6","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_SsNE","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_vNBa","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_3xm5","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_velZ","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_K3Dq","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_83Z8","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[],"columns":[{"dragId":"gw_6Ja_","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension","sort":"descending"}],"color":[{"dragId":"gw_8suy","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[{"dragId":"gw_Swt4","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"shape":[],"radius":[],"theta":[{"dragId":"gw_NIFP","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"longitude":[],"latitude":[],"geoId":[],"details":[{"dragId":"gw_E00c","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"filters":[{"dragId":"gw_MXz2","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["其他"]}}],"text":[{"dragId":"gw_AbRf","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}]},"layout":{"showActions":false,"showTableSummary":false,"stack":"normalize","interactiveScale":false,"zeroScale":true,"size":{"mode":"fixed","width":386,"height":362},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_naVb","name":"Chart 3"},{"config":{"defaultAggregated":true,"geoms":["arc"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_ampe","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_A0Sl","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_0hIL","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_Aat3","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_ZU_C","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_TSE-","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_yd3q","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_TBeh","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_B9He","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_OHrz","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_C9fi","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_NA6F","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"columns":[{"dragId":"gw_IzCC","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension","sort":"descending"}],"color":[{"dragId":"gw_lVpC","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[{"dragId":"gw_4T6w","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"shape":[],"radius":[],"theta":[{"dragId":"gw_TEam","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"longitude":[],"latitude":[],"geoId":[],"details":[{"dragId":"gw_0YVb","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"filters":[{"dragId":"gw_-wTp","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["其他"]}}],"text":[{"dragId":"gw_BmHj","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"fixed","width":500,"height":419},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_Ueyd","name":"Chart 4"},{"config":{"defaultAggregated":true,"geoms":["bar"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_A1N6","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_C2eR","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_uAOa","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_-neF","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_sXhq","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_nVFy","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_I0i6","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_5-gt","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_c6Wv","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_SD-F","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_kgN4","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_rd74","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_ycWo","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"columns":[{"dragId":"gw_rjv2","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension","sort":"descending"},{"dragId":"gw_jCUI","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension","sort":"descending"}],"color":[{"dragId":"gw_dkoh","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[{"dragId":"gw_vV8D","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"}],"filters":[{"dragId":"gw_uh_4","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["其他"]}}],"text":[{"dragId":"gw_2SnB","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"}]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"auto","width":624,"height":458},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_a34m","name":"Chart 5"},{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_92pJ","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_LO9-","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_n8ls","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_b8vg","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_hhRX","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_MjH1","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_igc7","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_LxJ3","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_MsHf","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_EnIO","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_t4VG","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_WaJ5","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_CtdE","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"columns":[{"dragId":"gw_iSwF","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_fgep","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"color":[{"dragId":"gw_Tfwc","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[],"shape":[{"dragId":"gw_eA-x","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[{"dragId":"gw_c7ZY","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"filters":[{"dragId":"gw_slXU","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["其他"]}}],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"auto","width":320,"height":200},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_S2bg","name":"Chart 7"}],"chart_map":{},"workflow_list":[{"workflow":[{"type":"filter","filters":[{"fid":"sub_type","rule":{"type":"not in","value":["其他"]}}]},{"type":"view","query":[{"op":"aggregate","groupBy":["sub_type"],"measures":[{"field":"sale_count","agg":"sum","asFieldKey":"sale_count_sum"}]}]}]},{"workflow":[{"type":"filter","filters":[{"fid":"main_type","rule":{"type":"not in","value":["其他"]}}]},{"type":"view","query":[{"op":"aggregate","groupBy":["main_type"],"measures":[{"field":"sale_count","agg":"sum","asFieldKey":"sale_count_sum"}]}]}]},{"workflow":[{"type":"filter","filters":[{"fid":"main_type","rule":{"type":"not in","value":["其他"]}}]},{"type":"view","query":[{"op":"aggregate","groupBy":["店名","main_type"],"measures":[{"field":"销售额","agg":"sum","asFieldKey":"销售额_sum"}]}]}]},{"workflow":[{"type":"filter","filters":[{"fid":"sub_type","rule":{"type":"not in","value":["其他"]}}]},{"type":"view","query":[{"op":"aggregate","groupBy":["店名","sub_type"],"measures":[{"field":"sale_count","agg":"sum","asFieldKey":"sale_count_sum"},{"field":"销售额","agg":"sum","asFieldKey":"销售额_sum"}]}]}]}],"version":"0.4.6"}"""
pyg.walk(data, spec=vis_spec)

分析结果：

Chart1：子类别的销量占比

从子类别销售量占比情况来看，底妆类、口红类在化妆品中销量最多，清洁类、化妆水、面霜类在护肤品中销量最多

Chart2：主类别的销量占比

从主类别销售量占比情况来看，护肤品的销量远大于化妆品

Chart3、Chart4：各品牌在主类别和子类别的销量和销量额情况

各品牌的化妆品、护肤品销量、销售情况均不一样，这与品牌的定位有关

主打化妆品的品牌，如蜜丝佛陀等销量较高

主打护肤品的品牌，如欧莱雅、佰草集等销量较高

*每个类别的平均价格、热度与销量的关系*

In [ ]:
vis_spec = r"""{"config":[{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_PjWF","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_sqYg","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_UMv1","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_L3ye","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_MXJ2","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_l6bx","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_Womx","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_RFQ0","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_uzPA","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_XsCw","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw__mlD","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_4oWZ","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"columns":[{"dragId":"gw_dacc","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension","sort":"descending"}],"color":[{"dragId":"gw_rM06","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"auto","width":320,"height":200},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_iit6","name":"Chart 1"},{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_POra","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_rXaZ","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_mf1Q","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_QRau","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_6Uc6","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_4Otc","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_pTzG","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_NNxK","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_PnZu","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_Arnf","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_nIVY","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_6TQq","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"columns":[{"dragId":"gw_fGIP","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension","sort":"descending"}],"color":[{"dragId":"gw_REqG","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"auto","width":320,"height":200},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_1b4M","name":"Chart 2"},{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_ZgMC","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_TcTU","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_ITOX","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_5jG9","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_psus","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_DJie","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_sWzk","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_CC_R","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"dimension","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_V-jp","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw__Q16","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_ZzCA","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_YQ1b","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"columns":[{"dragId":"gw_KI37","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"color":[{"dragId":"gw_HDeo","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[{"dragId":"gw_2QoU","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[{"dragId":"gw_bI7j","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["其他","底妆类","口红类","眼部彩妆","防晒类","修容类"]}}],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"fixed","width":660,"height":405},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_PqTL","name":"Chart 3"},{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_yrjq","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_mCTk","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw__KQN","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_CiRU","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_kF4n","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_eEI9","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_zxlv","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_kVbM","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_dEtD","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_jsAJ","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_2zdv","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_KaV4","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"columns":[{"dragId":"gw_qs4s","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"color":[{"dragId":"gw_a552","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[{"dragId":"gw_ky7l","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"mean"}],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[{"dragId":"gw_GN_Z","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["其他","清洁类","面霜类","化妆水","乳液类","眼部护理","精华类"]}}],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"fixed","width":580,"height":392},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_TUjP","name":"Chart 4"}],"chart_map":{},"workflow_list":[{"workflow":[{"type":"view","query":[{"op":"aggregate","groupBy":["main_type"],"measures":[{"field":"price","agg":"mean","asFieldKey":"price_mean"}]}]}]},{"workflow":[{"type":"view","query":[{"op":"aggregate","groupBy":["sub_type"],"measures":[{"field":"price","agg":"mean","asFieldKey":"price_mean"}]}]}]},{"workflow":[{"type":"filter","filters":[{"fid":"sub_type","rule":{"type":"not in","value":["其他","底妆类","口红类","眼部彩妆","防晒类","修容类"]}}]},{"type":"view","query":[{"op":"aggregate","groupBy":["sub_type"],"measures":[{"field":"sale_count","agg":"mean","asFieldKey":"sale_count_mean"},{"field":"comment_count","agg":"mean","asFieldKey":"comment_count_mean"},{"field":"price","agg":"mean","asFieldKey":"price_mean"}]}]}]},{"workflow":[{"type":"filter","filters":[{"fid":"sub_type","rule":{"type":"not in","value":["其他","清洁类","面霜类","化妆水","乳液类","眼部护理","精华类"]}}]},{"type":"view","query":[{"op":"aggregate","groupBy":["sub_type"],"measures":[{"field":"sale_count","agg":"mean","asFieldKey":"sale_count_mean"},{"field":"comment_count","agg":"mean","asFieldKey":"comment_count_mean"},{"field":"price","agg":"mean","asFieldKey":"price_mean"}]}]}]}],"version":"0.4.6"}"""
pyg.walk(data, spec=vis_spec)

分析结果：

Chart1：显示主类别产品的均价

护肤品均价高于化妆品

Chart2：显示子类别产品的均价

精华均价最高，防晒产品均价最低

Chart3：显示护肤类产品的销量和热度的关系

护肤类产品热度和销量呈正相关，价格越低销量越高热度越高

Chart3：显示化妆品类产品的销量和热度的关系

没有明显的关系，眼妆产品虽然价格低销量也不高热度不高，底妆产品价格最高但销量和热度都较高

*男性销售情况*

In [ ]:
vis_spec = r"""{"config":[{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_m07x","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_JA4C","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_zF_i","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_arNi","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_In3m","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_829D","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_bZrC","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_QQ1-","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_ZCnM","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_hXND","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_jXMD","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_-UAF","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}}],"columns":[{"dragId":"gw_mg3m","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension","sort":"descending"}],"color":[{"dragId":"gw_UqG4","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[{"dragId":"gw_HI1B","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["其他"]}},{"dragId":"gw_OzKO","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["其他"]}},{"dragId":"gw_zGhD","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["否"]}}],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"fixed","width":717,"height":435},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_B5EX","name":"Chart 1"},{"config":{"defaultAggregated":true,"geoms":["auto"],"coordSystem":"generic","limit":-1},"encodings":{"dimensions":[{"dragId":"gw_lAIh","fid":"id","name":"id","basename":"id","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_dyTH","fid":"title","name":"title","basename":"title","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_hI-f","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_Ezo-","fid":"sub_type","name":"sub_type","basename":"sub_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_QYHk","fid":"main_type","name":"main_type","basename":"main_type","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_y1u7","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension"},{"dragId":"gw_EVVo","fid":"day","name":"day","basename":"day","semanticType":"quantitative","analyticType":"dimension"},{"dragId":"gw_mea_key_fid","fid":"gw_mea_key_fid","name":"Measure names","analyticType":"dimension","semanticType":"nominal"}],"measures":[{"dragId":"gw_Cn9V","fid":"price","name":"price","basename":"price","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_iySW","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_ZVwr","fid":"comment_count","name":"comment_count","basename":"comment_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_bIX3","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_count_fid","fid":"gw_count_fid","name":"Row count","analyticType":"measure","semanticType":"quantitative","aggName":"sum","computed":true,"expression":{"op":"one","params":[],"as":"gw_count_fid"}},{"dragId":"gw_mea_val_fid","fid":"gw_mea_val_fid","name":"Measure values","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"rows":[{"dragId":"gw_NiiR","fid":"sale_count","name":"sale_count","basename":"sale_count","analyticType":"measure","semanticType":"quantitative","aggName":"sum"},{"dragId":"gw_MA1i","fid":"销售额","name":"销售额","basename":"销售额","analyticType":"measure","semanticType":"quantitative","aggName":"sum"}],"columns":[{"dragId":"gw_0Sd-","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension","sort":"descending"}],"color":[{"dragId":"gw_ER3P","fid":"店名","name":"店名","basename":"店名","semanticType":"nominal","analyticType":"dimension"}],"opacity":[],"size":[],"shape":[],"radius":[],"theta":[],"longitude":[],"latitude":[],"geoId":[],"details":[],"filters":[{"dragId":"gw_E1Te","fid":"是否男士专用","name":"是否男士专用","basename":"是否男士专用","semanticType":"nominal","analyticType":"dimension","rule":{"type":"not in","value":["否"]}}],"text":[]},"layout":{"showActions":false,"showTableSummary":false,"stack":"stack","interactiveScale":false,"zeroScale":true,"size":{"mode":"fixed","width":673,"height":587},"format":{},"geoKey":"name","resolve":{"x":false,"y":false,"color":false,"opacity":false,"shape":false,"size":false}},"visId":"gw_2rRt","name":"Chart 2"}],"chart_map":{},"workflow_list":[{"workflow":[{"type":"filter","filters":[{"fid":"sub_type","rule":{"type":"not in","value":["其他"]}},{"fid":"main_type","rule":{"type":"not in","value":["其他"]}},{"fid":"是否男士专用","rule":{"type":"not in","value":["否"]}}]},{"type":"transform","transform":[{"key":"gw_count_fid","expression":{"op":"one","params":[],"as":"gw_count_fid"}}]},{"type":"view","query":[{"op":"aggregate","groupBy":["店名","sub_type"],"measures":[{"field":"gw_count_fid","agg":"sum","asFieldKey":"gw_count_fid_sum"}]}]}]},{"workflow":[{"type":"filter","filters":[{"fid":"是否男士专用","rule":{"type":"not in","value":["否"]}}]},{"type":"view","query":[{"op":"aggregate","groupBy":["店名"],"measures":[{"field":"sale_count","agg":"sum","asFieldKey":"sale_count_sum"},{"field":"销售额","agg":"sum","asFieldKey":"销售额_sum"}]}]}]}],"version":"0.4.6"}"""
pyg.walk(data, spec=vis_spec)

分析结果：

Chart1：在男性使用的美妆品牌中，妮维雅数量最多，且主要为护肤品，化妆品很少

Chart2：在男性使用的美妆品牌中，妮维雅销量和销量额最多，第二第三分别为欧莱雅、相宜本草；

# 总结

1、由于商家在双十一提前预热，巨大的优惠力度和为了避免网络高峰，不少消费者选择提前消费，销量高峰出现在双十一前几天；双十一后3天商家持续打折优惠，消费者还保有购物余热，但远不如双十一之前。

2、价格较低的品牌带来更高的热度和销量，平价产品是大多数人的选择

3、护肤品的销量远远大于化妆品，护肤品中清洁类、面霜和化妆水销量较高，化妆品中底妆和口红较高

4、男性购买的产品集中在护肤品，其中妮维雅最受欢迎

# 建议

1、消费者在双十一当天的购买欲望较低，商家可以提前放出优惠刺激消费者提前消费。

2、消费者对产品价格和热度关注度较高，品牌可以适当调整产品价格并通过诸如网络社交平台推广的方式提高品牌热度。

3、对于男性消费者，品牌可以推送平价护肤品。